In [1]:
import os
from reportparse.reader.base import BaseReader
from reportparse.annotator.base import BaseAnnotator

reader = BaseReader.by_name('pymupdf')()
document = reader.read(input_path='./reportparse/asset/example.pdf')

document = BaseAnnotator.by_name("climate")().annotate(document=document)
document = BaseAnnotator.by_name("climate_commitment")().annotate(document=document)
document = BaseAnnotator.by_name("climate_sentiment")().annotate(document=document)
document = BaseAnnotator.by_name("climate_specificity")().annotate(document=document)
document = BaseAnnotator.by_name("llm_agg")().annotate(document=document)

[0310 21:57.08 @posthog.py:22]  INF  Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
/home/geoka/miniconda3/envs/rocm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0310 21:57.09 @SentenceTransformer.py:210]  INF  Use pytorch device_name: cuda
[0310 21:57.09 @SentenceTransformer.py:218]  INF  Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
[0310 21:57.11 @core.py:207]  INF  Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
[0310 21:57.12 @common.py:161]  INF  Downloaded file to /home/geoka/stanza_resources/resources.json
[0310 21:57.12 @core.py:271]  INF  Loading these mode

Starting storing in Chroma
Stored page 0
Stored page 1
Stored page 2


[0310 21:57.20 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_4
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_4_chunk_2
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_4_chunk_3
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_5
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_5_chunk_0
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_5_chunk_1
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_5_chunk_2
[0310 21:57.21 @local_persistent_hnsw.py:339]  WRN  Add of existing embedding ID: example.pdf_page_5_chunk_3


Stored page 3
Stored page 4
Stored page 5
Successfully stored all pages in chroma
Checking the first 1 pages for greenwashing


[0310 21:57.26 @_client.py:1025]  INF  HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
[0310 21:58.02 @chroma_annotator.py:179]  INF  Retrieving context from ChromaDB
[0310 21:58.02 @chroma_annotator.py:251]  INF  Calling LLM to verify claim with context


...

Error: https://links.duckduckgo.com/d.js?q=%22Contributing+to+a+Decarbonized+Society+Through+the+Decarbonization+Business%22&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-139721206602422698038450265276596038068&bing_market=wt-WT&ex=-2 202 Ratelimit, attempt 1/5
Retrying in 2 seconds...


[0310 21:58.37 @lib.rs:450]  INF  response: https://duckduckgo.com/?q=%22Contributing+to+a+Decarbonized+Society+Through+the+Decarbonization+Business%22 200
[0310 21:58.38 @lib.rs:450]  INF  response: https://links.duckduckgo.com/d.js?q=%22Contributing+to+a+Decarbonized+Society+Through+the+Decarbonization+Business%22&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-139721206602422698038450265276596038068&bing_market=wt-WT&ex=-2 200
[0310 21:58.38 @llm_aggregator.py:89]  INF  Calling LLM to verify claim with context


pdf
pdf
pdf
Could not find relevant sources.
Result is None


[0310 21:58.39 @_client.py:1025]  INF  HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AI message:  Statement: "Contributing to a Decarbonized Society Through the Decarbonization Business"

Result of the statement: PARTIALLY TRUE

Justification:

The Database Verdict provides strong confirmation that Hitachi aims to contribute to a decarbonized society through its decarbonization business, as evidenced by the company's long-term environmental targets and metrics for measuring progress. However, the Web Verdict's lack of information from the web means that we cannot verify or contradict this claim directly.

Given the Database Verdict's confirmation and the absence of contradictory evidence from the Web Verdict, I rely on the structured database to inform my conclusion. The justification provided by the Database Verdict highlights Hitachi's efforts to reduce CO2 emissions throughout its entire value chain, not just at its own business sites.

While the statement is partially true, it lacks context regarding environmental governance, which is crucial for achieving a decarb

In [6]:
if not os.path.exists('./results'):
    os.makedirs('./results')

# Save the full data as a JSON file
document.save('./results/example.pdf.json')
# Save the easy-to-use dataset as a CSV file
df = document.to_dataframe(level='block')

In [7]:
print(df)

        page_id              block_id block_layout_type  \
0    page_idx_0   -338662669616057431              text   
1    page_idx_0  -6141761642586137555              text   
2    page_idx_0  -2727539719969651667              text   
3    page_idx_0   1319615417483931358              text   
4    page_idx_0    504742589709628036              text   
..          ...                   ...               ...   
112  page_idx_5   3456605633321372939              text   
113  page_idx_5   3038824898059928662              text   
114  page_idx_5  -7266134418938778719              text   
115  page_idx_5   -256627500594058200              text   
116  page_idx_5  -4237186339689253465              text   

                                            block_bbox  \
0    (141.7324981689453, 68.15635681152344, 4192.91...   
1    (141.7324981689453, 299.5753173828125, 2012.86...   
2    (1110.2362060546875, 574.18701171875, 2062.130...   
3    (2173.228271484375, 299.5753173828125, 3104.15...   
4

In [ ]:
from pymongo import MongoClient
import json

# Load the JSON file
with open("./results/example.pdf.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Parses JSON into a Python dictionary or list

# Connect to MongoDB (default localhost:27017)
client = MongoClient("mongodb://localhost:27017/")
db = client["pdf_annotations"]  # Database name
collection = db["annotations"]  # Collection name

collection.insert_one(data) 

InsertOneResult(ObjectId('67cf44bd3d1cb875574caeb7'), acknowledged=True)

In [9]:
print(type(data))  # Should print <class 'list'> or <class 'dict'>


<class 'dict'>


In [10]:
import pprint

if isinstance(data, dict):  # If it's a dictionary
    pprint.pprint(list(data.items())[:2])  # Show first 2 key-value pairs
else:  # If it's a list, use slicing
    pprint.pprint(data[:2])


[('name', 'example.pdf'),
 ('pages',
  [{'annotations': [{'annotator': 'First pass',
                     'meta': {'info': 'Simple greenwashing detection'},
                     'value': 'Potential greenwashing claim: "Achieving a '
                              'Decarbonized Society"\n'
                              'Justification: The report claims to achieve a '
                              'decarbonized society, but it does not provide '
                              'specific details on how this goal is being met '
                              'or what concrete actions are being taken to '
                              'reduce greenhouse gas emissions.\n'
                              '\n'
                              'Potential greenwashing claim: "Contributing to '
                              'a Decarbonized Society Through the '
                              'Decarbonization Business"\n'
                              'Justification: This claim is vague and does not '
     

In [11]:
import json
from pymongo import MongoClient

with open("results/example.pdf.json", "r", encoding="utf-8") as file:
    data = json.load(file)

client = MongoClient("mongodb://localhost:27017/")
db = client["greenwashing_db"]
collection = db["reports"]

pdf_id = data["name"]
new_pages = data["pages"]

existing_doc = collection.find_one({"name": pdf_id})

if existing_doc:
    existing_pages = {page["num"] for page in existing_doc["pages"]}
    new_pages_to_insert = [page for page in new_pages if page["num"] not in existing_pages]

    if new_pages_to_insert:
        collection.update_one(
            {"name": pdf_id},
            {"$push": {"pages": {"$each": new_pages_to_insert}}}
        )
        print(f"Inserted {len(new_pages_to_insert)} new pages into {pdf_id}.")
    else:
        print(f"No new pages to insert for {pdf_id}.")
else:
    collection.insert_one(data)
    print(f"Inserted full document with {len(new_pages)} pages from {pdf_id}.")

result = collection.find_one({"name": pdf_id})
print(f"Final document contains {len(result['pages'])} pages.")


No new pages to insert for example.pdf.
Final document contains 6 pages.


[0311 12:28.30 @pymupdf_reader.py:205]  WRN  The "read" method received the "args" argument, which means any other optional arguments will be ignored.
[0311 12:28.30 @pymupdf_reader.py:163]  INF  Read page 0
[0311 12:28.30 @pymupdf_reader.py:37]  INF  	Load images from the page 0
[0311 12:28.30 @pymupdf_reader.py:186]  INF  	Read blocks
[0311 12:28.30 @pymupdf_reader.py:109]  INF  	Apply the sentence tokenization by SpaCy
[0311 12:28.30 @pymupdf_reader.py:163]  INF  Read page 1
[0311 12:28.30 @pymupdf_reader.py:37]  INF  	Load images from the page 1
[0311 12:28.30 @pymupdf_reader.py:186]  INF  	Read blocks
[0311 12:28.30 @pymupdf_reader.py:109]  INF  	Apply the sentence tokenization by SpaCy
[0311 12:28.30 @pymupdf_reader.py:163]  INF  Read page 2
[0311 12:28.30 @pymupdf_reader.py:37]  INF  	Load images from the page 2
[0311 12:28.30 @pymupdf_reader.py:186]  INF  	Read blocks
[0311 12:28.30 @pymupdf_reader.py:109]  INF  	Apply the sentence tokenization by SpaCy
[0311 12:28.31 @pymupdf_

        page_id              block_id block_layout_type  \
0    page_idx_0  -4564257573540477673              text   
1    page_idx_0  -3157173936657975544              text   
2    page_idx_0  -7227624621501901522              text   
3    page_idx_0   7881401589232902542              text   
4    page_idx_0   6867980831404772257              text   
..          ...                   ...               ...   
112  page_idx_5  -7725027972843899705              text   
113  page_idx_5  -1097985037442280917              text   
114  page_idx_5   4286364780799275154              text   
115  page_idx_5   6192970086282277809              text   
116  page_idx_5   4055846838829822740              text   

                                            block_bbox  \
0    (141.7324981689453, 68.15635681152344, 4192.91...   
1    (141.7324981689453, 299.5753173828125, 2012.86...   
2    (1110.2362060546875, 574.18701171875, 2062.130...   
3    (2173.228271484375, 299.5753173828125, 3104.15...   
4

In [ ]:
jsonfile='/Users/geoka/Documents/pdf_annotations.annotations.json'

import json
#load the json file
with open(jsonfile, 'r') as file:
    data = json.load(file)

# make it a csv as well
import pandas as pd
df = pd.json_normalize(data['pages'])
df.to_csv(jsonfile.replace('.json', '.csv'), index=False)